# Post-process vasca_pipe results
Make catalog file of selected sources only, associate and get store additional info for listed sources

In [1]:
region_name = "ALL_10-800" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
region_cat_fname = region_fname.replace(".fits","_cat.fits")

do_cat = True                       # Recreate catalog file from vasca_pipe output region
do_cross_match = True            # Does source matching with CDS, typically for all (~10k) catalog sources
do_variable_addition = True        # Adds variables of matched sources to tt_sources
do_LombScargle = True        # Run LombScargle, typically for all (~10k) catalog sources
do_src_prep = True               # Gets additional info (e.g. SED from Vizer) for sources listes in srcs_ids list below. This is slow, and therefore only for the sources listed in srcs_ids below

srcs_ids = [535864 ,451644 ,541266 ,581995 ,187856 ,166179 ,34658 ,426363 ,151796 ,259271 ,419001 ,25273 ,]
srcs_ids = [357455] # Massive pulsating WD

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
logger.enable("vasca")

# Do catalog file

if do_cat:
    rg = Region()
    rg.load_from_fits(region_fname) 
    rc = rg.get_region_catalog()
else:
    rc = Region()
    rc.load_from_fits(region_cat_fname)

#Cross match to SIMBAD and GAIA
if do_cross_match:
    rc.cross_match_cds(query_table="J/MNRAS/508/3877/maincat", vizier_columns=["*", "meanAV", "minAV", "maxAV"])
    
    rc.cross_match_cds(query_table="simbad", overwrite=False)
    rc.cross_match_cds(overwrite=True)
    rc.tt_gaiadr3.rename_column("E_BP-RP_","E_BP-RP")
    display(rc.tt_gaiadr3)

    #Crossmatch with GFCAT
    tt_cat = Table.read("./resources/GFCAT/hlsp_gfcat_galex_imaging_object-cat_fuv-nuv_v1_table.ecsv")
    rc.add_table(tt_cat,"tt_gfcat")
    rc.cross_match(dist_max=1.5 * uu.arcsec, dist_s2n_max=None, cat_table_name="tt_gfcat",cat_id_name="gfcat_src_id",cat_name="gfcat", src_table_name="tt_sources")

    #rc.cross_match_cds(query_table="I/358/varisum",vizier_columns=["*", "NG", "dGmag"],)

#Add variables
if do_variable_addition:
    rc.add_column("tt_gaiadr3", "Gmag_abs")
    sel_plx = rc.tt_gaiadr3["RPlx"]>4
    rc.tt_gaiadr3["Gmag_abs"][sel_plx]=rc.tt_gaiadr3["Gmag"][sel_plx] + 5.0 * np.log10(rc.tt_gaiadr3["Plx"][sel_plx]) - 10.0
    rc.tt_gaiadr3["Plx_dist"] = rc.tt_gaiadr3["Plx"].quantity.to(uu.parsec, equivalencies=uu.parallax())

    # Add normalized excess
    rc.tt_sources["flux_ne"]=np.sqrt(rc.tt_sources["flux_nxv"])

    # Add object groups
    add_ogrp(rc.tt_simbad)
    add_ogrp(rc.tt_gaiadr3,provenance="GAIA")

    src_coords = SkyCoord(ra=rc.tt_sources["ra"].quantity, dec=rc.tt_sources["dec"].quantity, frame='icrs')
    #print(src_coords.to_string('hmsdms')) #'decimal'
    src_names = [] 
    for coord in src_coords:
        scoord = coord.to_string('hmsdms', precision=1, alwayssign=True, pad=False, fields=3, format=None)
        scoord = scoord.replace("s ","").replace("s","").replace("d","").replace("m","").replace("h","")
        src_names.append(("1UVA J"+scoord).replace("J+","J"))
    rc.add_column("tt_sources", "src_name", col_data=src_names)

if do_LombScargle:
    rc.set_LombScargle(nbins_min=20)

#Write out all changes to region
rc.write_to_fits(region_cat_fname, overwrite=True)

#Prepare sources, including Vizier SED.
if do_src_prep:
    for src_id in srcs_ids:
        print(f"*********** Creating Source Table ID {src_id} *****************")
        tc_src = rc.get_src_from_id(src_id, load_from_file = False)
print("Done")

2024-02-05 10:49:53.412 | DEBUG    | vasca.tables:load_from_fits:328 - Loading file with name './vasca_pipeline/ALL_10-800/region_ALL_10-800.fits'
2024-02-05 10:49:53.422 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_fields'
2024-02-05 10:49:53.433 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_filters'
2024-02-05 10:49:53.439 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_visits'
2024-02-05 10:49:53.445 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_sources'
2024-02-05 10:49:54.202 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_detections'
2024-02-05 10:49:56.929 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_detections'
2024-02-05 10:49:57.111 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_sources'
2024-02-05 10:49:57.523 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_src_id_map'
2024-02-05 10:49:57.595 | DEBUG    | vasca.region:

_q,ra,dec,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000,PQSO,PGal,PSS,RPlx,VarFlag,o_Gmag,RFRP,RFBP,AG,E_BP-RP,rg_src_id,match_distance,gaiadr3_match_id
,deg,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg,,,,,,,,,mag,mag,,arcsec,
int32,float64,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,str12,str15,str19,int32,str10,str16,str17,str21,float64,float64,float64,float64,float64,float64,str13,int16,float32,float32,float32,float32,int32,float64,int32
1,150.19472313100,2.06790658862,3836069006998531584,2.2872,3.3137,--,--,--,--,--,--,--,--,95.82888,1.689,20.733625,93.75,16.38,20.408556,130.9,9.985,19.455320,0.953236,--,--,--,--,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,110481501947501974,1237651753997304055,--,,,J100046.74+020404.4,--,N6SS000981,,,,150.19472313100,2.06790658862,0.001869,0.000724,0.996765,--,NOT_AVAILABLE,79,13.11,5.725,--,--,41,0.6956587221689177,0
2,150.14558890142,2.04305474716,3836046123412862208,1.5776,2.0453,--,--,--,--,--,--,--,--,90.12499,2.676,20.800253,37.12,20.52,21.414429,59.56,10.83,20.310590,1.103838,--,--,--,--,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,110451501456232172,1237651736280760581,--,,,J100034.94+020234.9,--,N6SS004829,,,,150.14558890142,2.04305474716,0.001265,0.000013,0.997987,--,NOT_AVAILABLE,43,5.497,1.809,--,--,68,0.6420858569707824,1
4,150.10199844257,2.10546555318,3836257710681556224,0.5228,0.5439,-0.0737,0.5331,0.702,-0.580,0.607,0.396,0.641,1.065,231.69009,2.456,19.775099,160.4,10.57,19.825287,132.1,14.81,19.445590,0.379698,--,--,--,--,1,0,0,0,0,0,1,0,0,0,0,--,--,--,--,--,--,110521501020297033,1237651753997238486,--,,,J100024.44+020619.5,--,N6SV003245,,,,150.10200102167,2.10546379217,1.000000,0.000000,0.000000,-0.1383,VARIABLE,182,8.917,15.18,--,--,87,1.015315761207988,2
7,215.06519632330,52.62189251083,1607957522794289024,0.1805,0.2101,-0.4327,0.2279,0.356,0.190,0.223,0.302,0.256,1.036,311.79894,1.36,19.452680,178.4,9.809,19.709816,189.3,7.18,19.055190,0.654627,--,--,--,--,1,0,0,0,0,0,1,0,0,0,0,--,--,--,--,--,--,171142150651746989,1237661385994862739,--,,,J142015.62+523718.5,--,N5MV001218,,,,215.06519493360,52.62189116990,1.000000,0.000000,0.000000,-1.8990,VARIABLE,482,26.36,18.19,--,--,302,0.26795579682948123,3
8,215.04268959491,52.67487914728,1607958480571152256,0.8988,0.7715,-1.1739,0.6843,0.532,-0.092,1.013,0.524,0.894,1.079,127.05163,2.726,20.427416,98.51,10.98,20.354832,127.2,7.357,19.486479,0.868353,--,--,--,--,1,0,0,0,0,0,1,0,0,0,0,--,--,--,--,--,--,171212150426430532,1237661385994862858,--,,,J142010.22+524029.6,--,N5MV001750,,,,215.04269027125,52.67487681872,0.999765,0.000220,0.000015,-1.7155,VARIABLE,283,17.29,8.972,--,--,314,0.5654019830257911,4
9,214.94309900678,52.54622023872,1607951746062355328,0.2074,0.2337,0.1023,0.2656,0.429,-0.239,0.256,-0.357,0.272,0.966,233.47251,0.8803,19.766777,140.9,7.083,19.965982,148.7,6.695,19.317305,0.648678,--,--,--,--,1,0,0,0,0,0,1,0,0,0,0,--,--,--,--,--,--,171052149430476033,1237661385994862854,--,,,J141946.35+523246.3,--,N5MV000642,,,,214.94310075250,52.54622182347,1.000000,0.000000,0.000000,0.3850,VARIABLE,523,22.21,19.9,--,--,336,0.5780435001189599,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

2024-02-05 10:50:14.778 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_gfcat'
2024-02-05 10:50:14.779 | INFO     | vasca.tables:cross_match:1180 - Cross matching table tt_sources
/tmp/ipykernel_13033/1616927836.py:60: RuntimeWarning: invalid value encountered in sqrt
  rc.tt_sources["flux_ne"]=np.sqrt(rc.tt_sources["flux_nxv"])
2024-02-05 10:50:15.547 | DEBUG    | vasca.region:set_LombScargle:847 - Running LombScargle..
2024-02-05 10:50:28.931 | DEBUG    | vasca.region:set_LombScargle:849 - Starting source loop
2024-02-05 10:50:41.873 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_lombscargle'
2024-02-05 10:50:41.904 | INFO     | vasca.tables:write_to_fits:222 - Writing file with name './vasca_pipeline/ALL_10-800/region_ALL_10-800_cat.fits'
2024-02-05 10:50:41.913 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_fields'
2024-02-05 10:50:41.928 | DEBUG    | vasca.tables:write_to_fits:262 - Writing table 'tt_filters'
2024-02-05 10:50:41.932 |

*********** Creating Source Table ID 357455 *****************


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'